In [119]:
%reset -f
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU 
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,Conv2D,Lambda, LSTM
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


from tensorflow.keras.models import Model
from tensorflow.keras import Input

from tensorflow.keras.datasets.mnist import load_data

# Libraires traitements données avant de s'entrainer
from sklearn.utils import shuffle
from sklearn.preprocessing import minmax_scale

# Libraires classique
import numpy as np
import matplotlib.pyplot as plt
import random

# pandas
import pandas as pd


In [120]:
#fix random seed for reproductability
seed=7 #afin de toujours générer les mêmes nombres aléatoires 
np.random.seed(seed)
n_samples= 128
latent_dim = 100
Niter = 10000
lr1 = 0.001
lr2 = 0.0001
time_steps=20

># Data loading

In [121]:
data = np.load('dataset.npy')
data

array([[-7.83001781e-01,  3.22366881e+00, -1.05017734e+00, ...,
         1.67788565e+00,  7.35204220e-01,  2.71000004e+00],
       [ 5.82768381e-01,  1.26537287e+00,  3.31506103e-01, ...,
         1.15772474e+00, -4.77928221e-01,  1.75999999e+00],
       [ 1.37821400e+00,  4.43961716e+00, -9.84977603e-01, ...,
         2.13262892e+00,  6.27748430e-01,  2.00000000e+00],
       ...,
       [ 2.79434514e+00,  1.27241573e+01, -1.20561590e+01, ...,
         2.22915149e+00, -1.34131098e+00,  2.64000010e+00],
       [ 1.50969052e+00,  2.43533373e+00,  9.29814339e+00, ...,
         2.11973572e+00,  1.23632317e-02,  2.89000010e+00],
       [ 3.88355637e+00,  5.68180180e+00,  9.57663894e-01, ...,
         3.54852915e+00,  4.55817890e+00,  3.64000010e+00]])

In [122]:
X_train = np.zeros((1300,20,58))

In [123]:
for k in range(1300):
    X_train[k] = data[k:k+20]
X_train

array([[[-0.78300178,  3.22366881, -1.05017734, ...,  1.67788565,
          0.73520422,  2.71000004],
        [ 0.58276838,  1.26537287,  0.3315061 , ...,  1.15772474,
         -0.47792822,  1.75999999],
        [ 1.378214  ,  4.43961716, -0.9849776 , ...,  2.13262892,
          0.62774843,  2.        ],
        ...,
        [-0.2789402 ,  1.29782939, -1.0897876 , ...,  1.33365953,
         -1.27552748,  1.29999995],
        [-2.12925005,  4.85793686, -5.69753218, ...,  2.17745233,
         -1.30720818,  2.78999996],
        [-1.88002646,  7.23364449, -2.1368928 , ...,  4.62183857,
          1.95700967,  6.26000023]],

       [[ 0.58276838,  1.26537287,  0.3315061 , ...,  1.15772474,
         -0.47792822,  1.75999999],
        [ 1.378214  ,  4.43961716, -0.9849776 , ...,  2.13262892,
          0.62774843,  2.        ],
        [ 0.58032346,  1.3730427 , -1.10211813, ...,  2.05557275,
         -0.35282296,  1.90999997],
        ...,
        [-2.12925005,  4.85793686, -5.69753218, ...,  

In [124]:
X_test = np.zeros((300,20,58))

In [125]:
for k in range(300):
    X_test[k] = data[1300+k:1320+k]
X_test

array([[[ 0.66731173,  2.1513412 ,  0.81895751, ...,  1.0958178 ,
          1.39506781,  1.42999995],
        [-0.82742786,  1.76194823, -0.65765673, ...,  0.62389284,
          0.85026491,  1.03999996],
        [ 1.80078661,  1.56249452, -0.11735369, ...,  0.58890164,
          0.26700503,  0.73000002],
        ...,
        [-0.96894461,  0.70695865,  0.53420514, ...,  0.51747811,
          0.6483317 ,  0.46000001],
        [-0.4739005 ,  1.1457895 ,  0.50022173, ...,  0.5089469 ,
          0.16595395,  0.69      ],
        [-0.63676047,  1.23704135,  0.16092461, ...,  0.63711751,
          0.53860092,  0.91000003]],

       [[-0.82742786,  1.76194823, -0.65765673, ...,  0.62389284,
          0.85026491,  1.03999996],
        [ 1.80078661,  1.56249452, -0.11735369, ...,  0.58890164,
          0.26700503,  0.73000002],
        [ 0.26358193,  2.20727944, -0.1984784 , ...,  0.81833446,
          0.02651113,  1.09000003],
        ...,
        [-0.4739005 ,  1.1457895 ,  0.50022173, ...,  

### Normalisation des données 

In [126]:
mu = np.mean(X_train,axis=(0,1))
sigma = np.std(X_train,axis=(0,1))
print(f'mu: {mu}\nsigma: {sigma}')

mu: [-0.1166307   3.14107658 -0.04011315  2.19112573  0.04719889  2.11582746
  0.02624354  2.54621493  0.07429094  1.23820196 -0.01572099  2.24752555
 -0.00557022  2.20901654 -0.01861671  1.61384461 -0.01126602  1.89319177
  0.05958467  2.84782318 -0.03806358  1.9540121  -0.18908329  3.23813642
 -0.02768227  2.52090402  0.03598356  1.56131556 -0.07911     3.77706841
  0.01744245  1.42699366  0.0074425   2.98766887  0.04880688  1.31753112
  0.09707499  2.34556616 -0.00612595  1.32470551  0.03678727  2.17588177
 -0.02135862  2.10687817  0.10359043  1.07402255 -0.00786091  0.74332766
 -0.02410385  2.79227415 -0.01289601  1.83316786 -0.03323821  2.43276501
 -0.02841867  1.6673663   0.02884083  1.73431769]
sigma: [1.62988301 2.78400071 1.45870699 3.26338427 1.39930173 3.63911729
 1.47192143 2.76279564 1.05051598 1.75849385 1.38401079 4.76760418
 1.48732143 1.8757085  1.19424924 1.42194571 1.24501492 2.28177373
 1.53669297 3.62945491 1.33105755 3.0105356  1.91583087 3.32571249
 1.49259626 3.

In [127]:
X_train_normalized = (X_train - mu)/sigma

#### Vérification

In [128]:
print(f'moy: {np.mean(X_train_normalized,axis=(0,1))}\necart_type: {np.std(X_train_normalized,axis=(0,1))}')

moy: [-4.84825855e-17 -1.05285438e-14  3.49318864e-16  3.30052865e-15
  1.55179288e-16 -2.11950329e-15 -1.46966307e-16 -3.76078655e-15
 -5.67186522e-16  1.70295188e-15 -2.58895469e-17  5.83333488e-15
  5.40640182e-16  1.77787699e-15  1.78078038e-16  4.56137265e-16
  2.49492734e-16  1.99477080e-15  2.22516450e-16  3.66600767e-15
  2.30681927e-16  6.56003030e-15  5.76718160e-17  1.17944543e-15
  2.62670227e-16 -5.06533277e-15  6.80536823e-16 -2.84699401e-15
 -7.04137603e-18 -1.11548804e-15 -4.47142323e-16  1.60620128e-15
 -7.42440297e-17  2.83689845e-15  9.34192894e-16  8.37330205e-15
 -1.14438373e-18 -4.35211482e-15  8.46256825e-16  3.10512300e-16
 -9.35486731e-16 -1.95964185e-15 -2.56828746e-16 -6.27547160e-15
  4.26154838e-17 -3.11180141e-15  3.88982116e-16  4.76230593e-15
 -1.22639078e-16 -1.50573998e-16  1.14090361e-16 -3.10294525e-16
 -1.12375921e-16  5.82095910e-15 -8.91466388e-17 -9.31561238e-15
  4.13568752e-17  2.64109247e-16]
ecart_type: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

># Creation du modèle

In [129]:
model = Sequential()

In [130]:
model.add(LSTM(units=58,activation='tanh'))

In [131]:
model.add(Dense(units=1))

In [132]:
model.compile(optimizer='adam',loss='mse')

In [134]:
model.fit(X_train,epochs=10,batch_size=20)

Epoch 1/10


NotImplementedError: in user code:

    C:\ProgramData\anaconda3\envs\Tensorflow_GPU_0\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\anaconda3\envs\Tensorflow_GPU_0\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\anaconda3\envs\Tensorflow_GPU_0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\anaconda3\envs\Tensorflow_GPU_0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\anaconda3\envs\Tensorflow_GPU_0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args,